# The effect of COVID-19 on deliveredKwh
We designed two machine learning-based model (SVM-based) that to predict the value of "dailyDeleivered". 

We monitored the prediction error of the pre and post COVID data to observe the effect of COVID-19 on the prediction error. 
We also observed the sign of the prediction error to see whether the predicted dailyDelivered values are greater or less than the actula values.
The following code and results are for a dataset that contains data of residential premises.

The prediction error for post-covid data is greater than the one for pre-covid data.

In [1]:
import numpy as np
import pandas as pd
import sklearn
#import sklearn.datasets as ds
import sklearn.model_selection as ms
import sklearn.svm as svm
from numpy import genfromtxt
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize
from operator import add
#rom keras.backend import tensorflow_backend as K

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
def temporalize(arr,win_size,step_size,features,add_index=0):
  """
  arr: any 2D array whose columns are distinct variables and 
    rows are data records at some timestamp t
  win_size: size of data window (given in data points)
  step_size: size of window step (given in data point)
  
  Note that step_size is related to window overlap (overlap = win_size - step_size), in 
  case you think in overlaps."""
  #
  dataFrameTimeseries=pd.DataFrame()
  #
  w_list = list()
  n_records = arr.shape[0]
  remainder = (n_records - win_size) % step_size 
  num_windows = 1 + int((n_records - win_size - remainder) / step_size)
  for k in range(num_windows):
    w_list.append(arr[k*step_size:win_size-1+k*step_size+1])
    #
    #convert the matrix to data frame
    dataFrameTemp=pd.DataFrame(data=arr[k*step_size:win_size-1+k*step_size+1], columns=features)
    dataFrameTemp["timeseriesId"]=k+add_index
    dataFrameTimeseries=pd.concat([dataFrameTimeseries,dataFrameTemp])
    #
  return np.array(w_list),dataFrameTimeseries


In [3]:
def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            pass
        
        try:
            import unicodedata
            unicodedata.numeric(s)
            return True
        except (TypeError, ValueError):
            pass
        return False 

In [4]:
def plot_(h_axis,v1_axis,v2_axis, title="Daily Delivered (kWh)"):
    plt.rcParams.update({'font.size': 15})
    plt.tight_layout()
    plt.figure(figsize=(30,3))
    plt.plot(h_axis, v1_axis, color="blue",marker="o", label="actual")
    plt.plot(h_axis,v2_axis, color="red",marker="o",label="predicted")
    plt.xticks(h_axis[0::31],rotation=90)
    plt.title(title)
    plt.show()

In [5]:
# split into train and test sets
def split_(ds, dimension=2):
    train_size = 365
    test_size = len(ds) - train_size
    train=[]
    test=[]
    if dimension==1:
        train, test = ds[0:train_size], ds[train_size:len(ds)]
    else:
        train, test = ds[0:train_size,:], ds[train_size:len(ds),:]
    return train,test


In [7]:
def evaluation_(y, yhat):
    mse = metrics.mean_squared_error(y, yhat)
    rmse = np.sqrt(mse)
    r2 = metrics.r2_score(y,yhat)
    #er = (y - yhat)/y) * 100 
    
    print("Results of sklearn.metrics:")
    print("RMSE:", rmse)
    print("R-Squared:", r2)
    #print("Error rate:" , er)

In [17]:
def svm_poly(X,Y,trainig_percentage=0.76):
    
    model = svm.SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1, coef0=1)
    
    #train on precovid data and test on all dataset
    Xtrain,Xtest=split_(X)
    Ytrain,Ytest=split_(Y,1)
    
    model.fit(Xtrain, Ytrain)
    
    #prediction error on pre-covid data (training data)
    yhat1=model.predict(Xtrain)
    prediction_error1=np.square(Ytrain-yhat1).mean(axis=None).ravel()
    print("prediction error on pre-covid data")
    print("**********************************")
    print(prediction_error1)
    
    #prediction error on post-covid data (testing data)
    yhat2 = model.predict(Xtest)
    prediction_error2=np.square(Ytest-yhat2).mean(axis=None).ravel()
    print("prediction error on post-covid data")
    print("**********************************")
    print(prediction_error2)
    
    print("Error sign (actual-prediction) for post covid data: If Positive--> actual is greater than predicted")
    print("**********************************")
    print((Ytest-yhat2).mean(axis=None))
    
    #predict the entire data
    yhat=model.predict(X)
    
    dataset["dailyDelivered"] = pd.to_datetime(dataset["dailyDelivered"], format='%Y-%m')

    h_axis=dataset["time"]
    v1_axis=dataset["dailyDelivered"]
    v2_axis=yhat.flatten()
    #temp=np.zeros(win_size)
    #v2_axis=np.concatenate([temp,v2_axis])
    plot_(h_axis,v1_axis,v2_axis)
    
    evaluation_(Y, yhat)

In [15]:
def svm_rbf(X,Y,trainig_percentage=0.76):
    
    model = svm.SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
    
    #train on precovid data and test on all dataset
    Xtrain,Xtest=split_(X)
    Ytrain,Ytest=split_(Y,1)
    
    model.fit(Xtrain, Ytrain)
    
    #prediction error on pre-covid data (training data)
    yhat1=model.predict(Xtrain)
    prediction_error1=np.square(Ytrain-yhat1).mean(axis=None).ravel()
    print("prediction error on pre-covid data")
    print("**********************************")
    print(prediction_error1)
    
    #prediction error on post-covid data (testing data)
    yhat2 = model.predict(Xtest)
    prediction_error2=np.square(Ytest-yhat2).mean(axis=None).ravel()
    print("prediction error on post-covid data")
    print("**********************************")
    print(prediction_error2)
    
    print("Error sign (actual-prediction) for post covid data: If Positive--> actual is greater than predicted")
    print("**********************************")
    print((Ytest-yhat2).mean(axis=None))
    
    #predict the entire data
    yhat=model.predict(X)
    
    dataset["dailyDelivered"] = pd.to_datetime(dataset["dailyDelivered"], format='%Y-%m')

    h_axis=dataset["time"]
    v1_axis=dataset["dailyDelivered"]
    v2_axis=yhat.flatten()
    #temp=np.zeros(win_size)
    #v2_axis=np.concatenate([temp,v2_axis])
    
    plot_(h_axis,v1_axis,v2_axis)
    
    evaluation_(Y, yhat)

In [19]:
dataset["dailyDelivered"]

0     1970-01-01
1     1970-01-01
2     1970-01-01
3     1970-01-01
4     1970-01-01
5     1970-01-01
6     1970-01-01
7     1970-01-01
8     1970-01-01
9     1970-01-01
10    1970-01-01
11    1970-01-01
12    1970-01-01
13    1970-01-01
14    1970-01-01
15    1970-01-01
16    1970-01-01
17    1970-01-01
18    1970-01-01
19    1970-01-01
20    1970-01-01
21    1970-01-01
22    1970-01-01
23    1970-01-01
24    1970-01-01
25    1970-01-01
26    1970-01-01
27    1970-01-01
28    1970-01-01
29    1970-01-01
         ...    
609   1970-01-01
610   1970-01-01
611   1970-01-01
612   1970-01-01
613   1970-01-01
614   1970-01-01
615   1970-01-01
616   1970-01-01
617   1970-01-01
618   1970-01-01
619   1970-01-01
620   1970-01-01
621   1970-01-01
622   1970-01-01
623   1970-01-01
624   1970-01-01
625   1970-01-01
626   1970-01-01
627   1970-01-01
628   1970-01-01
629   1970-01-01
630   1970-01-01
631   1970-01-01
632   1970-01-01
633   1970-01-01
634   1970-01-01
635   1970-01-01
636   1970-01-

In [10]:
def svm_sig(X,Y,trainig_percentage=0.76):
    
    model = svm.SVR(kernel='sigmoid', C=100, gamma=0.1, epsilon=.1)
    
    #train on precovid data and test on all dataset
    Xtrain,Xtest=split_(X)
    Ytrain,Ytest=split_(Y,1)
    
    model.fit(Xtrain, Ytrain)
    
    #prediction error on pre-covid data (training data)
    yhat1=model.predict(Xtrain)
    prediction_error1=np.square(Ytrain-yhat1).mean(axis=None).ravel()
    print("prediction error on pre-covid data")
    print("**********************************")
    print(prediction_error1)
    
    #prediction error on post-covid data (testing data)
    yhat2 = model.predict(Xtest)
    prediction_error2=np.square(Ytest-yhat2).mean(axis=None).ravel()
    print("prediction error on post-covid data")
    print("**********************************")
    print(prediction_error2)
    
    print("Error sign (actual-prediction) for post covid data: If Positive--> actual is greater than predicted")
    print("**********************************")
    print((Ytest-yhat2).mean(axis=None))
    
    #predict the entire data
    yhat=model.predict(X)

    h_axis=dataset["time"]
    v1_axis=dataset["dailyDelivered"]
    v2_axis=yhat.flatten()
    #temp=np.zeros(win_size)
    #v2_axis=np.concatenate([temp,v2_axis])
    plot_(h_axis,v1_axis,v2_axis)
    
    evaluation_(Y, yhat)

In [11]:
#set window sizes and their overlap
win_size=10
overlap=win_size-1

#read data
dataset = pd.read_csv('sourceData/GS_Energy.csv', engine='python')

#preprocess data
dataset=dataset.fillna(-1)


for column in dataset.columns:
    if is_number(dataset.iloc[1][column]) and column!="idx":
        min_max=MinMaxScaler(feature_range=(0, 1))
        dataset[[column]]=min_max.fit_transform(dataset[[column]])
       
#set variables based on the data attributes    
n_features=len(dataset.columns.values)
dailyDeliveredIndex=np.where(dataset.columns.values=="dailyDelivered")
dailyDeliveredIndex=dailyDeliveredIndex[0][0]

X_ = dataset[["idx", "Temp", "isWeekend"]].values
y_ = dataset[["idx", "dailyDelivered"]].values

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype bool were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [18]:
svm_poly(X_[:,1:], y_[:,1:])

prediction error on pre-covid data
**********************************
[0.0591095]
prediction error on post-covid data
**********************************
[0.08005854]
Error sign (actual-prediction) for post covid data: If Positive--> actual is greater than predicted
**********************************
-0.08897844715783423


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<Figure size 432x288 with 0 Axes>

ValueError: view limit minimum -35957.916777027676 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

<Figure size 2160x216 with 1 Axes>

Results of sklearn.metrics:
RMSE: 0.12086821025178499
R-Squared: 0.6232231771815235


In [ ]:
svm_rbf(X_[:,1:], y_[:,1:])